# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#I seperated all random points from only points over land and ran seperate analysis on both
#The idea being that if temperatures do increase towrd the equator it would be the same
#for both ocean and land temperatures.
#Max temperatures appear to be around the 23rd latitude, which is likely due to the time of year.
#Additional panel data and more precise definitions of warmer would likely be needed to fully verify.
#Simple linear methods show no strong correlation between latitude and temp, but linear models are weak for weather data
#The graphs however show some trends that might be uncovered with other methods

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import openweathermapy.core as owm
import seaborn as sns
import json
from scipy.stats import linregress, shapiro, normaltest

# Import API key
from api_keys import api_key
settings = {"units": "metric", "appid": api_key}
# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

#Weather API Variables
w_url = 'https://api.openweathermap.org/data/2.5/weather'

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [13]:
# List for holding lat_lngs and cities
lat_lngs = []
city_name = []
country=[]
clouds =[]
date=[]
humidity=[]
max_temp =[]
min_temp=[]
wind_spd =[]
lat =[]
lon =[]

# Create a set of random lat and lng combinations
# Hi Dayanara - the graphs look way better when the size below is big, but it takes one second
#    per call in the API. So it can take a while to run when it looks it's best.
#    set size = 1000 for speed, but look at the saved graphs to see what the larger set looks like.
lats = np.random.uniform(low=-90.000, high=90.000, size=500)#set to 1000 for speed
lngs = np.random.uniform(low=-180.000, high=180.000, size=500)#set to 1000 for speed
lat_lngs = zip(lats, lngs)

# Doing the call srtaight to owm to skip the missing cities stuff
for i,lat_lng in enumerate(lat_lngs):
    req = owm.get_current(lat_lng,**settings)
    if req['name']=='':
        city_name.append('Ocean')
    else:
        city_name.append(req['name'])
    if req['name']=='':
        country.append('Ocean')
    else:
        country.append(req['sys']['country'])
    clouds.append(req['clouds']['all'])
    date.append(req['dt'])
    humidity.append(req['main']['humidity'])
    max_temp.append(req['main']['temp_max'])
    min_temp.append(req['main']['temp_min'])
    wind_spd.append(req['wind']['speed'])
    lat.append(lat_lng[0])
    lon.append(lat_lng[1])


# Print the city count to confirm sufficient count
weather_dict = {
    'city':city_name,
    'country':country,
    'cloudiness':clouds,
    'date':date,
    'humidity':humidity,
    'max_temp':max_temp,
    'min_temp':min_temp,
    'wind_speed':wind_spd,
    'lat':lat,
    'lon':lon
    
}

all_weather_df = pd.DataFrame(weather_dict)
land_weather_df = all_weather_df.loc[all_weather_df['city']!="Ocean"]
print('count of all datapoints:' + str(len(all_weather_df)))
print('count of land datapoints:' + str(len(land_weather_df)))
all_weather_df.head()

count of all datapoints:500
count of land datapoints:148


,city,country,cloudiness,date,humidity,max_temp,min_temp,wind_speed,lat,lon
0,Kangavar,IR,0,1567548011,26,17.38,17.38,1.16,34.299583,47.940029
1,Ocean,Ocean,95,1567548012,82,7.67,7.67,14.10,-53.306129,-126.121669
2,Ocean,Ocean,100,1567548012,90,0.57,0.57,10.37,75.645301,115.336776
3,Ocean,Ocean,100,1567548013,94,-0.53,-0.53,5.53,-54.305839,-38.943239
4,Ocean,Ocean,68,1567548013,71,28.17,28.17,2.99,34.010940,148.940305


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [16]:
all_weather_df.to_html('weather_data.html')

UnicodeEncodeError: 'charmap' codec can't encode character '\u012b' in position 17773: character maps to <undefined>

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
all_weather_df.head()

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

In [ ]:
plt.scatter(all_weather_df['lat'], all_weather_df['max_temp'])
plt.title('Latitude vs Max Daily Temp All Points')
plt.ylabel('Max Temp (C$^\circ$)')
plt.xlabel('Latitude')
plt.grid(True, alpha=0.50)
plt.savefig('Lat vs Max Temp All')

In [ ]:
plt.scatter(land_weather_df['lat'], land_weather_df['max_temp'])
plt.title('Latitude vs Max Daily Temp Land Points')
plt.ylabel('Max Temp (C$^\circ$)')
plt.xlabel('Latitude')
plt.grid(True, alpha=0.50)
plt.savefig('Lat vs Max Temp Land')

In [ ]:
plt.scatter(all_weather_df['lat'], all_weather_df['min_temp'])
plt.title('Latitude vs Min Daily Temp Land Points')
plt.ylabel('Min Temp (C$^\circ$)')
plt.xlabel('Latitude')
plt.grid(True, alpha=0.50)
plt.savefig('Lat vs Min Temp All')

In [ ]:
plt.scatter(land_weather_df['lat'], land_weather_df['min_temp'])
plt.title('Latitude vs Min Daily Temp Land Points')
plt.ylabel('Min Temp (C$^\circ$)')
plt.xlabel('Latitude')
plt.grid(True, alpha=0.50)
plt.savefig('Lat vs Min Temp Land')

#### Latitude vs. Humidity Plot

In [ ]:
all_weather_df = all_weather_df[all_weather_df['humidity']<=100]
plt.scatter(all_weather_df['lat'], all_weather_df['humidity'])
plt.title('Latitude vs Humidity')
plt.ylabel('Humidity %')
plt.xlabel('Latitude')
plt.grid(True, alpha=0.50)
plt.savefig('Lat vs Humidity All')

In [ ]:
land_weather_df = land_weather_df[land_weather_df['humidity']<=100]
plt.scatter(land_weather_df['lat'], land_weather_df['humidity'])
plt.title('Latitude vs Humidity')
plt.ylabel('Humidity %')
plt.xlabel('Latitude')
plt.grid(True, alpha=0.50)
plt.savefig('Lat vs Humidity Land')

#### Latitude vs. Cloudiness Plot

In [ ]:
plt.scatter(all_weather_df['lat'], all_weather_df['cloudiness'])
plt.title('Latitude vs Cloudiness All Point')
plt.ylabel('Cloudiness %')
plt.xlabel('Latitude')
plt.grid(True, alpha=0.50)
plt.savefig('Lat vs Clouds All')

In [ ]:
plt.scatter(land_weather_df['lat'], land_weather_df['cloudiness'])
plt.title('Latitude vs Cloudiness Land Points')
plt.ylabel('Cloudiness %')
plt.xlabel('Latitude')
plt.grid(True, alpha=0.50)
plt.savefig('Lat vs Clouds Land')

#### Latitude vs. Wind Speed Plot

In [ ]:
plt.scatter(all_weather_df['lat'], all_weather_df['wind_speed'])
plt.title('Latitude vs Wind Speed All Points')
plt.ylabel('Wind Speed m/s')
plt.xlabel('Latitude')
plt.grid(True, alpha=0.50)
plt.savefig('Lat vs Wind Speed All')

In [ ]:
plt.scatter(land_weather_df['lat'], land_weather_df['wind_speed'])
plt.title('Latitude vs Wind Speed Land Points')
plt.ylabel('Wind Speed m/s')
plt.xlabel('Latitude')
plt.grid(True, alpha=0.50)
plt.savefig('Lat vs Wind Speed Land')

In [ ]:
#You can see that the middle is warmer, but generallt further north of the equator. Since we're looking at 
#current weather data this is likely becasue of the time of year (i.e. the sun is up north right now)
#if we had more time some kind time weighted average would be cool
color = land_weather_df['max_temp']
plt.scatter(land_weather_df['lon'], land_weather_df['lat'], c=color )
plt.savefig('Lat vs World Map Land')

In [ ]:
#shows our data are normal, so regular methods are cool
stat,p = normaltest(all_weather_df['max_temp'])
p

In [ ]:
#test all the data points, using abolute value of lat should tell us if geting closer to the equator matters
#bascially it's a maybe and we should probably use something other than linear models
test_vars = ['max_temp', 'min_temp', 'humidity', 'cloudiness','wind_speed']
for t_var in test_vars:
    results = linregress(abs(all_weather_df['lat']), all_weather_df[t_var])
    print(results)
    

In [ ]:
#test all the data points, using abolute value of lat should tell us if geting closer to the equator matters
#bascially it's a maybe and we should probably use something other than linear models
print(len(land_weather_df))
for t_var in test_vars:
    results = linregress(abs(land_weather_df['lat']), land_weather_df[t_var])
    print(results)

In [ ]:
all_weather_df.to_json('weather_data.json')